In [ ]:
# import internal modules
import file_path_management as fpath
import public_library as plib
import extract_info
import parameters as params

In [ ]:
# import packages
import csv
import pandas as pd
import PyPDF2
import requests
import time
import os
import random
from requests.auth import HTTPProxyAuth
from bs4 import BeautifulSoup
import numpy as np
from selenium import webdriver
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.service import Service
from selenium.common.exceptions import TimeoutException, WebDriverException, NoSuchElementException
import os
import re
from lxml import etree

In [ ]:
# extract text from given .pdf file
def pdf2text(pdf_path, text_path, page_start, page_end):
    try:
        # creating a pdf reader object
        reader = PyPDF2.PdfReader(pdf_path)
        
        # printing number of pages in pdf file
        page_max = len(reader.pages)
        
        # getting a specific page from the pdf file
        text = ""
        
        for i in range(page_start, page_end + 1):
            # print(i)
            page = reader.pages[i]
            text = text + "".join(page.extract_text().splitlines())

        with open(text_path, "w") as f:
            f.write(text)
        f.close()

        return True
    except:
        return False
# --------------------start of test code--------------------
# ind = 1
# pdf_file_name = str(index) + ".pdf"
# pdf_folder = fpath.litera_pdf_folder
# pdf_path = os.path.join(pdf_folder, pdf_file_name)

# text_folder_path = fpath.litera_text_folder
# text_file_name = pdf_file_name.split(".pdf")[0] + ".txt"
# text_path = os.path.join(text_folder_path, text_file_name)

# start_page = 0
# end_page = 1
# if not pdf2text(pdf_path, text_path, start_page, end_page):
#     print("Error: pdf2text() failed")
# ---------------------end of test code---------------------

In [ ]:
# download pdf to specified folder given pdf_url and ind
def download_pdf(pdf_url, pdf_folder):  
    url = pdf_url
    if url != url:
        raise Exception("pdf_url is np.nan")
    else:
        source = url.split("://")[1].split("/")[0]
    
    # pdf_path = os.path.join(pdf_folder, file_name)
      
    # get the pdf content
    # options = webdriver.ChromeOptions()
    # options.add_experimental_option('prefs', {
    # "download.default_directory": pdf_folder, #Change default directory for downloads
    # "download.prompt_for_download": False, #To auto download the file
    # "download.directory_upgrade": True,
    # "download.open_pdf_in_system_reader": False,
    # "plugins.always_open_pdf_externally": True #It will not show PDF directly in chrome
    # })
    # driver = webdriver.Chrome(options=options)
    # # response = driver.get(pdf_url)
    # driver.navigate().to(url)
    # time.sleep(5)
    if source == "www.sciencedirect.com":
        os.environ['WDM_LOG'] = '0'
        options = Options()
        options.add_argument('--headless')
        driver = webdriver.Chrome()
        driver.get(url)
        element = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, "//li[@class='ViewPDF']/a")))
        element.click()
        time.sleep(10)
    else: 
        options = webdriver.ChromeOptions()
        options.add_experimental_option('prefs', {
        "download.default_directory": pdf_folder, #Change default directory for downloads
        "download.prompt_for_download": False, #To auto download the file
        "download.directory_upgrade": True,
        "plugins.always_open_pdf_externally": True #It will not show PDF directly in chrome
        })
        driver = webdriver.Chrome(options=options)
        response = driver.get(url)
        time.sleep(10)
    driver.close()

    # try:
    #     with open(pdf_path, 'wb') as pdf_object:
    #         pdf_object.write(response.content)
    #         driver.close()
    #         return True
    # except:
    #     print(f'Failed downloading PDF:', url)
    #     driver.close()
    #     # print(f'HTTP response status code: {response.status_code}')
    #     return False
# --------------------start of test code--------------------
# # pdf_url = 'https://www.sciencedirect.com/science/article/pii/S0896627320300052/pdfft?md5=3f0648c6385e6fae3a5a73b053903014&pid=1-s2.0-S0896627320300052-main.pdf'
# # pdf_url = "https://www.sciencedirect.com/science/article/pii/S1053811909013299?via%3Dihub"
# # pdf_url = "https://linkinghub.elsevier.com/retrieve/pii/S1053811909013299"
# # pdf_url = "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1181753/pdf/jphysiol00456-0559.pdf"
# pdf_url = "https://www.sciencedirect.com/science/article/pii/S0896627320300052?via%3Dihub"

# # wiley.com
# # pdf_url = "https://onlinelibrary.wiley.com/doi/pdf/10.1002/cne.903130106"
# # pdf_url = "https://onlinelibrary.wiley.com/doi/pdfdirect/10.1002/cne.903130106"
# # pdf_url = "https://onlinelibrary.wiley.com/doi/pdf/10.1002/cne.902770204"
# # pdf_url = "https://onlinelibrary.wiley.com/doi/pdfdirect/10.1002/cne.902770204"
# # pdf_url = "https://onlinelibrary.wiley.com/doi/pdf/10.1002/cne.902970309"
# # pdf_url = "https://onlinelibrary.wiley.com/doi/pdfdirect/10.1002/cne.902970309"

# # physiology.org
# # pdf_url = "https://journals.physiology.org/doi/pdf/10.1152/jn.2001.85.1.219"

# # springer.com
# # pdf_url = "https://link.springer.com/content/pdf/10.1007/BF00236173.pdf"

# # cell.com
# # pdf_url = "https://www.cell.com/neuron/pdf/S0896-6273(09)00170-6.pdf"

# # jneurosci.org
# # pdf_url = "https://www.jneurosci.org/content/jneuro/22/18/8117.full.pdf"
# # pdf_url, code = plib.get_final_redirected_url(pdf_url)
# # print(pdf_url)
# ind = 10
# file_name = str(ind) + ".pdf"
# pdf_folder = fpath.litera_pdf_folder
# download_pdf(pdf_url, pdf_folder)
# # rename_pdf(file_name, pdf_folder, time_to_wait=60)
# ---------------------end of test code---------------------

In [ ]:
def download_from_www_ncbi_nlm_nih_gov(url, ind, pdf_folder):
    try:
        file_name = str(ind) + ".pdf"
        response = requests.get(url, headers=plib.headers)
        # response = requests.get(url, stream=True, allow_redirects=True, headers=plib.headers)
        
        # download the .pdf file to the pdf_file_path folder
        # write content in pdf file
        pdf_path = os.path.join(pdf_folder, file_name)
        
        if response.status_code == 200:
            with open(pdf_path, 'wb') as pdf_object:
                pdf_object.write(response.content)
            print(f'Successfully downloaded PDF:', ind)
        else:
            print(f'Failed downloading PDF:' + 'pdf_url')
            print(f'HTTP response status code: {response.status_code}')
    except:
        print(f'Failed downloading PDF:' + 'pdf_url')
# --------------------start of test code--------------------
# pdf_url = "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6577493/pdf/jneuro_14_5_2485.pdf"
# ind = 10
# file_name = str(ind) + ".pdf"
# pdf_folder = "/home/hou/myProjects/litera_pdfs"
# download_from_www_ncbi_nlm_nih_gov(pdf_url, ind, pdf_folder)
# # rename_pdf(file_name, pdf_folder, time_to_wait=60)
# ---------------------end of test code---------------------

In [ ]:
# def download_from_ELSEVIER(doi, file_to_save_to):
#     url = 'http://api.elsevier.com/content/article/doi:'+doi+'?view=FULL'
#     headers = {
#         'X-ELS-APIKEY': "63f58b8b10cbc1bc923011c01c6301bb",
#         'Accept': 'application/pdf'
#     }
#     r = requests.get(url, stream=True, headers=headers)
#     if r.status_code == 200:
#         for chunk in r.iter_content(chunk_size=1024*1024):
#             file_to_save_to.write(chunk)
#         return True

# # doi_list = pd.read_excel('list.xls')
# # doi_list.columns = ['DOIs']
# count = 0
# # for doi in doi_list['DOIs']:
# doi = "10.1016/0006-8993(95)01338-5"
# # pdf = "10"
# ind = 10
# file_name = str(ind) + ".pdf"
# # if not os.path.exists(f'path/{pdf}.pdf'):
# with open(file_name, 'wb') as pdf_object:
#     get_pdf(doi, pdf_object)
# pdf_object.close

In [ ]:
# download pdf
def download_pdf(pdf_url, pdf_source, pdf_folder):
    for website in params.website_pdfs:
        func = None
        if website in pdf_source:
            # Get the function name by replacing "." with "_" and use globals() to call it
            func_name = "download_from_" + website.replace(".", "_")
            # print(func_name)
            func = globals().get(func_name)
            # print(func)
            break
    if func != None:
        # print(func)
        func(pdf_url, pdf_folder)
    else:
        print("The given url is not from a supported website: ", pdf_url)
        raise Exception("Function does not exist for website:", pdf_url)

# rename downloaded pdf
def rename_pdf(ind, pdf_folder, time_to_wait=60):
    newname = str(ind) + ".pdf"
    time_counter = 0
    filename = max([f for f in os.listdir(pdf_folder)], key=lambda xa :   os.path.getctime(os.path.join(pdf_folder,xa)))
    while '.part' in filename:
        time.sleep(1)
        time_counter += 1
        if time_counter > time_to_wait:
            raise Exception('Waited too long for file to download')
    filename = max([f for f in os.listdir(pdf_folder)], key=lambda xa :   os.path.getctime(os.path.join(pdf_folder,xa)))
    os.rename(os.path.join(pdf_folder, filename), os.path.join(pdf_folder, newname))

# download and rename pdf
def download_and_rename_pdf(pdf_url, ind, pdf_folder):  
    if pdf_url != pdf_url:
        raise Exception("pdf_url is np.nan")
    else:
        pdf_source = pdf_url.split("://")[1].split("/")[0]
    download_pdf(pdf_url, pdf_source, pdf_folder)
    rename_pdf(ind, pdf_folder, time_to_wait=60)
    print(f"Downloaded {ind}.pdf")
# --------------------start of test code--------------------
# pdf_url = "https://journals.physiology.org/doi/pdf/10.1152/jn.2001.85.1.219"
# ind = 10
# pdf_folder = "/home/hou/myProjects/litera_pdfs"
# download_and_rename_pdf(pdf_url, ind, pdf_folder)
# ---------------------end of test code---------------------
# https://journals.physiology.org/doi/10.1152/jn.2001.85.1.219

In [ ]:
# wiley.com
def wiley_com(url):
    # initialize
    info = {
        "doi": np.nan,
        "pmid": np.nan,
        "pmcid": np.nan,
        "title": np.nan,
        "abstract": np.nan,
        "keywords": np.nan,
        "pdf_link": np.nan
    }

    # set up the webdriver
    os.environ['WDM_LOG'] = '0'
    options = Options()
    options.add_argument('--headless')
    driver = webdriver.Firefox(options=options)

    # load the webpage
    error_label = 0
    while(error_label == 0):
        try:
            driver.get(url)
            time.sleep(5)
            error_label = 1
        except:
            print("Extracting content from:" + url + " failed, retrying... This might take longer than 5 minutes...")
            time.sleep(5*60)
            error_label = 0
    
    # doi
    try:
        doi = driver.find_element(By.XPATH, "//a[@class='epub-doi']").text.split("doi.org/")[1]
        doi = doi.strip()
    except:
        doi = np.nan
    if doi == doi:
        doi = doi.lower()

    # pmid, pmcid
    pmid = np.nan
    pmcid = np.nan

    # title
    try:
        title = driver.find_element(By.XPATH, "//h1[@class='citation__title']").text
        title = title.strip()
    except:
        try:
            title = driver.find_element(By.XPATH, "//h2[@class='citation__title']").text
            title = title.strip()
        except:
            title = np.nan
    
    # abstract
    try:
        abstract = ""
        try:
            elems = driver.find_element(By.XPATH, "//h3[text()='Abstract' or text()='ABSTRACT' or text()='Summary' or text()='SUMMARY']").find_element(By.XPATH, "following-sibling::div").find_elements(By.XPATH, 'p')
        except:
            elems = driver.find_element(By.XPATH, "//h2[text()='Abstract' or text()='ABSTRACT' or text()='Summary' or text()='SUMMARY']").find_element(By.XPATH, "following-sibling::div").find_elements(By.XPATH, 'p')                
        for elem in elems:
            abstract = abstract + elem.text + " "
        abstract = abstract.strip()
    except:
        abstract = np.nan
    
    # keywords
    keywords = np.nan

    # pdf_link
    try:
        pdf_link = driver.find_element(By.XPATH, "//a[@title='ePDF']").get_attribute('href')
        pdf_link = pdf_link.strip()
    except:
        pdf_link = np.nan

    driver.quit()

    info = {
        "doi": doi,
        "pmid": pmid,
        "pmcid": pmcid,
        "title": title,
        "abstract": abstract,
        "keywords": keywords,
        "pdf_link": pdf_link
    }

    return info
# --------------------start of test code--------------------
# # url = "https://onlinelibrary.wiley.com/doi/abs/10.1002/cne.901980111"
# url = "https://onlinelibrary.wiley.com/doi/10.1002/cne.21155"
# # url = "https://onlinelibrary.wiley.com/doi/10.1002/(SICI)1096-9861(19981019)400:2%3C271::AID-CNE8%3E3.0.CO;2-6"
# # url = "https://onlinelibrary.wiley.com/doi/10.1002/cne.902360304"
# # url = "https://onlinelibrary.wiley.com/doi/10.1002/cne.902820107"
# # url = "https://onlinelibrary.wiley.com/doi/10.1002/(SICI)1096-9861(19990726)410:2%3C211::AID-CNE4%3E3.0.CO;2-X"
# # url = "https://onlinelibrary.wiley.com/doi/10.1002/cne.902940314"
# # url = "https://onlinelibrary.wiley.com/doi/10.1002/cne.901990104"
# # url = "https://onlinelibrary.wiley.com/doi/10.1002/(SICI)1096-9861(19981019)400:2%3C271::AID-CNE8%3E3.0.CO;2-6"
# # url = "https://onlinelibrary.wiley.com/doi/10.1002/cne.902360304"
# # url = "https://onlinelibrary.wiley.com/doi/10.1002/cne.24389"
# # url = "https://onlinelibrary.wiley.com/doi/10.1002/(SICI)1096-9861(19960805)371:4%3C513::AID-CNE2%3E3.0.CO;2-7"
# # url = "https://nyaspubs.onlinelibrary.wiley.com/doi/full/10.1196/annals.1300.030"
# # url = "https://onlinelibrary.wiley.com/doi/10.1002/cne.23436"
# # url = "https://onlinelibrary.wiley.com/doi/10.1002/9780470513545.ch4"
# info = wiley_com(url)
# print(info["doi"])
# print(info["pmid"])
# print(info["pmcid"])
# print(info["title"])
# print(info["abstract"])
# print(info["keywords"])
# print(info["pdf_link"])
# ---------------------end of test code---------------------

In [ ]:
# neurology.org
def neurology_org(url):
    os.environ['WDM_LOG'] = '0'
    options = Options()
    options.add_argument('--headless')
    
    # load the webpage
    error_label = 0
    while(error_label == 0):
        try:
            driver = webdriver.Firefox(options=options)
            driver.get(url)
            time.sleep(5)
            error_label = 1
        except:
            print("Extracting content from:" + url + " failed, retrying... This might take longer than 5 minutes...")
            time.sleep(5*60)
            error_label = 0
    
    try:
        doi = driver.find_element(By.XPATH, "//span[contains(@class, 'highwire-cite-metadata-doi highwire-cite-metadata')]").text.split("doi.org/")[1]
    except:
        doi = np.nan
    if doi == doi:
        doi = doi.lower()
    pmid = np.nan
    pmcid = np.nan
    
    # title
    try:
        title = driver.find_element(By.XPATH, "//h1[@id='page-title']").text
        title = title.strip()
    except:
        title = np.nan
    
    # abstract
    try:
        abstract = ""
        elems = driver.find_element(By.XPATH, "//div[@class='section abstract']").find_elements(By.XPATH, "following-sibling::p")
        for elem in elems:
            abstract = abstract + elem.text + " "
        abstract = abstract.strip()
    except:
        abstract = np.nan
    
    # keywords
    keywords = np.nan

    # pdf_link
    try:
        pdf_link = driver.find_element(By.XPATH, "//a[@class='link-icon']").get_attribute('href')
        pdf_link = pdf_link.strip()
    except:
        pdf_link = np.nan

    driver.quit()

    info = {
        "doi": doi,
        "pmid": pmid,
        "pmcid": pmcid,
        "title": title,
        "abstract": abstract,
        "keywords": keywords,
        "pdf_link": pdf_link
    }
    driver.quit

    return info
# --------------------start of test code--------------------
# url = "https://n.neurology.org/content/64/6/1014.short"
# # url = "https://n.neurology.org/content/64/6/1014"
# # url = "https://n.neurology.org/content/64/6/1014"
# # url = "https://n.neurology.org/content/32/10/1198"
# # url = "https://n.neurology.org/content/22/9/998"
# info = neurology_org(url)
# print(info["doi"])
# print(info["pmid"])
# print(info["pmcid"])
# print(info["title"])
# print(info["abstract"])
# print(info["keywords"])
# print(info["introduction"])
# print(info["pdf_link"])
# ---------------------end of test code---------------------

In [ ]:
# www.biorxiv.org
def www_biorxiv_org(url):
    os.environ['WDM_LOG'] = '0'
    options = Options()
    options.add_argument('--headless')
    
    # load the webpage
    error_label = 0
    while(error_label == 0):
        try:
            driver = webdriver.Firefox(options=options)
            driver.get(url)
            time.sleep(5)
            error_label = 1
        except:
            print("Extracting content from:" + url + " failed, retrying... This might take longer than 5 minutes...")
            time.sleep(5*60)
            error_label = 0
    
    # doi
    try:
        doi = driver.find_element(By.XPATH, "//span[@class='highwire-cite-metadata-doi highwire-cite-metadata']").find_element(By.TAG_NAME, "span").text.split("doi.org/")[1]
    except:
        doi = np.nan
    
    # pmid, pmcid
    pmid = np.nan
    pmcid = np.nan

    # title
    try:
        title = driver.find_element(By.XPATH, "//h1[@id='page-title']").text
    except:
        title = np.nan
    
    # abstract
    try:
        abstract = ""
        abstract = driver.find_element(By.XPATH, "//div[@class='section abstract')]").find_elements(By.XPATH, 'following-sibling::p')
        for elem in abstract:
            abstract = abstract + elem.text + " "
    except:
        abstract = np.nan
    
    # keywords
    keywords = np.nan
    
    # pdf_link
    try:
        pdf_link = driver.find_element(By.XPATH, "//a[@class='biorxiv_pdf_dl_link')]").get_attribute("href")
    except:
        pdf_link = np.nan

    driver.quit()

    info = {
        "doi": doi,
        "pmid": pmid,
        "pmcid": pmcid,
        "title": title,
        "abstract": abstract,
        "keywords": keywords,
        "pdf_link": pdf_link
    }
    driver.quit

    return info
# --------------------start of test code--------------------
# url = "https://www.biorxiv.org/content/10.1101/398917v1.abstract"
# # url = "https://www.biorxiv.org/content/10.1101/2022.05.18.492367v2"
# # url = "https://www.biorxiv.org/content/10.1101/398917v1"
# # url = "https://www.biorxiv.org/content/10.1101/2021.02.01.429141v2"
# info = www_biorxiv_org(url)
# print(info["doi"])
# print(info["pmid"])
# print(info["pmcid"])
# print(info["title"])
# print(info["abstract"])
# print(info["keywords"])
# print(info["pdf_link"])
# ---------------------end of test code---------------------

In [ ]:
# europepmc.org
def europepmc_org(url):
    os.environ['WDM_LOG'] = '0'
    options = Options()
    options.add_argument('--headless')
    
    # load the webpage
    error_label = 0
    while(error_label == 0):
        try:
            driver = webdriver.Firefox(options=options)
            driver.get(url)
            time.sleep(5)
            error_label = 1
        except:
            print("Extracting content from:" + url + " failed, retrying... This might take longer than 5 minutes...")
            time.sleep(5*60)
            error_label = 0
    
    # doi
    try:
        doi = driver.find_element(By.XPATH, "//span[@class='metadata--doi']").find_element(By.TAG_NAME, "a").text.split("doi.org/")[1]
    except:
        doi = np.nan
    if doi == doi:
        doi = doi.lower()
    
    # pmid, pmcid
    try:
        pmid = driver.find_element(By.XPATH, "//span[contains(@class, 'metadata--pmid')]").text.split("PMID: ")[1]
    except:
        pmid = np.nan
    try:
        pmcid = driver.find_element(By.XPATH, "//span[contains(@class, 'metadata--pmcid')]").text.split("PMCID: ")[1]
    except:
        pmcid = np.nan
    
    # title
    try:
        title = driver.find_element(By.XPATH, "//h1[@class='article-metadata-title']").text
    except:
        title = np.nan
    
    # abstract
    try:
        abstract = driver.find_element(By.XPATH, "//div[contains(@class, 'abstract')]").text
    except:
        abstract = np.nan
    
    # keywords
    keywords = np.nan
    
    # pdf_link
    pdf_link = "://europepmc.org/"

    driver.quit()

    info = {
        "doi": doi,
        "pmid": pmid,
        "pmcid": pmcid,
        "title": title,
        "abstract": abstract,
        "keywords": keywords,
        "pdf_link": pdf_link
    }
    driver.quit

    return info
# --------------------start of test code--------------------
# url = "https://europepmc.org/article/MED/37298594"
# # url = "https://europepmc.org/article/med/8784824"
# # url = "https://europepmc.org/article/med/823649"
# # url = "https://europepmc.org/article/med/4220147"
# info = europepmc_org(url)
# print(info["doi"])
# print(info["pmid"])
# print(info["pmcid"])
# print(info["title"])
# print(info["abstract"])
# print(info["keywords"])
# print(info["pdf_link"])
# ---------------------end of test code---------------------

In [ ]:
# iovs.arvojournals.org
def iovs_arvojournals_org(url):
    # initialize
    info = {
        "doi": np.nan,
        "pmid": np.nan,
        "pmcid": np.nan,
        "title": np.nan,
        "abstract": np.nan,
        "keywords": np.nan,
        "pdf_link": np.nan
    }

    # set up the webdriver
    os.environ['WDM_LOG'] = '0'
    options = Options()
    options.add_argument('--headless')
    driver = webdriver.Firefox(options=options)

    # load the webpage
    error_label = 0
    while(error_label == 0):
        try:
            driver.get(url)
            time.sleep(5)
            error_label = 1
        except:
            print("Extracting content from:" + url + " failed, retrying... This might take longer than 5 minutes...")
            time.sleep(5*60)
            error_label = 0
    
    # doi
    try:
        elems = driver.find_element(By.XPATH, "//div[@class='ww-citation large-view-only']").find_elements(By.TAG_NAME, "span")
        for elem in elems:
            if "doi.org" in elem.text:
                doi = elem.text.split("doi.org/")[1]
                break
        doi = doi.strip()
    except:
        doi = np.nan
    if doi == doi:
        doi = doi.lower()

    # pmid, pmcid
    pmid = np.nan
    pmcid = np.nan

    # title
    try:
        title = driver.find_element(By.XPATH, "//div[@class='wi-article-title article-title-main']").text
        title = title.strip()
    except:
        title = np.nan
    
    # abstract
    try:
        abstract = ""
        elems = driver.find_element(By.XPATH, "//section[@class='abstract']").find_element(By.TAG_NAME, "p")
        for elem in elems:
            abstract = abstract + elem.text + " "
        abstract = abstract.strip()
    except:
        abstract = np.nan
    
    # keywords
    keywords = np.nan

    # pdf_link
    try:
        pdf_link = driver.find_element(By.XPATH, "//a[@id='pdfLink']").get_attribute('data-article-url')
        pdf_link = pdf_link.strip()
    except:
        pdf_link = np.nan

    driver.quit()

    info = {
        "doi": doi,
        "pmid": pmid,
        "pmcid": pmcid,
        "title": title,
        "abstract": abstract,
        "keywords": keywords,
        "pdf_link": pdf_link
    }

    return info
# --------------------start of test code--------------------
# url = "https://iovs.arvojournals.org/article.aspx?articleid=2124655"
# # url = "https://iovs.arvojournals.org/article.aspx?articleid=2659608"
# info = iovs_arvojournals_org(url)
# print(info["doi"])
# print(info["pmid"])
# print(info["pmcid"])
# print(info["title"])
# print(info["abstract"])
# print(info["keywords"])
# print(info["pdf_link"])
# ---------------------end of test code---------------------

In [ ]:
# royalsocietypublishing.org
def royalsocietypublishing_org(url):
    os.environ['WDM_LOG'] = '0'
    options = Options()
    options.add_argument('--headless')
    
    # load the webpage
    error_label = 0
    while(error_label == 0):
        try:
            driver = webdriver.Firefox(options=options)
            driver.get(url)
            time.sleep(5)
            error_label = 1
        except:
            print("Extracting content from:" + url + " failed, retrying... This might take longer than 5 minutes...")
            time.sleep(5*60)
            error_label = 0
    
    # doi
    try:
        doi = driver.find_element(By.XPATH, "//a[contains(@class, 'epub-section__doi__text')]").text.split("doi.org/")[1]
    except:
        doi = np.nan
    
    # pmid, pmcid
    pmid = np.nan
    pmcid = np.nan

    # title
    try:
        title = driver.find_element(By.XPATH, "//h1[@class='citation__title']").text
        title = title.strip()
    except:
        title = np.nan
    
    # abstract
    try:
        abstract = ""
        elems = driver.find_element(By.XPATH, "//div[@class='abstractSection abstractInFull']").find_elements(By.XPATH, 'p')
        for elem in elems:
            abstract = abstract + elem.text + " "
        abstract = abstract.strip()
    except:
        abstract = np.nan
    
    # keywords
    keywords = np.nan

    # pdf_link
    pdf_link = "://royalsocietypublishing.org/"

    driver.quit()

    info = {
        "doi": doi,
        "pmid": pmid,
        "pmcid": pmcid,
        "title": title,
        "abstract": abstract,
        "keywords": keywords,
        "pdf_link": pdf_link
    }
    driver.quit

    return info
# --------------------start of test code--------------------
# url = "https://royalsocietypublishing.org/doi/abs/10.1098/rstb.2002.1171"
# # url = "https://royalsocietypublishing.org/doi/10.1098/rspb.1972.0087"
# # url = "https://royalsocietypublishing.org/doi/10.1098/rstb.1984.0021"
# info = royalsocietypublishing_org(url)
# print(info["doi"])
# print(info["pmid"])
# print(info["pmcid"])
# print(info["title"])
# print(info["abstract"])
# print(info["keywords"])
# print(info["pdf_link"])
# ---------------------end of test code---------------------

In [ ]:
# psychiatryonline.org
def psychiatryonline_org(url):
    os.environ['WDM_LOG'] = '0'
    options = Options()
    options.add_argument('--headless')
    
    # load the webpage
    error_label = 0
    while(error_label == 0):
        try:
            driver = webdriver.Firefox(options=options)
            driver.get(url)
            time.sleep(5)
            error_label = 1
        except:
            print("Extracting content from:" + url + " failed, retrying... This might take longer than 5 minutes...")
            time.sleep(5*60)
            error_label = 0
    
    # doi
    try:
        doi = driver.find_element(By.XPATH, "//a[contains(@class, 'epub-section__doi__text')]").text.split("doi.org/")[1]
    except:
        doi = np.nan
    
    # pmid, pmcid
    pmid = np.nan
    pmcid = np.nan
    
    # title
    try:
        title = driver.find_element(By.XPATH, "//h1[@class='citation__title']").text
        title = title.strip()
    except:
        title = np.nan
    
    # abstract
    try:
        abstract = ""
        elems = driver.find_element(By.XPATH, "//div[@class='abstractSection abstractInFull']").find_elements(By.XPATH, 'p')
        for elem in elems:
            abstract = abstract + elem.text + " "
        abstract = abstract.strip()
    except:
        abstract = np.nan
    
    # keywords
    keywords = np.nan

    # pdf_link
    try:
        elems = driver.find_element(By.XPATH, "//ul[@title='coolBar__drop rlist w-slide--list cloned hidden-xs hidden-sm js--open']").find_elements(By.TAG_NAME, "li")
        for elem in elems:
            t = elem.find_element(By.TAG_NAME, "a").find_element(By.TAG_NAME, "span")
            if "PDF" in t:
                pdf_link = elem.find_element(By.TAG_NAME, "a").get_attribute("href")
                break
        pdf_link = pdf_link.strip()
    except:
        pdf_link = np.nan

    driver.quit()

    info = {
        "doi": doi,
        "pmid": pmid,
        "pmcid": pmcid,
        "title": title,
        "abstract": abstract,
        "keywords": keywords,
        "pdf_link": pdf_link
    }
    driver.quit

    return info
# --------------------start of test code--------------------
# url = "https://ajp.psychiatryonline.org/doi/full/10.1176/appi.ajp.161.5.896"
# # url = "https://ajp.psychiatryonline.org/doi/10.1176/ajp.156.11.1709"
# # url = "https://ajp.psychiatryonline.org/doi/full/10.1176/appi.ajp.158.9.1411"
# info = psychiatryonline_org(url)
# print(info["doi"])
# print(info["pmid"])
# print(info["pmcid"])
# print(info["title"])
# print(info["abstract"])
# print(info["keywords"])
# print(info["pdf_link"])
# ---------------------end of test code---------------------

In [ ]:
# direct.mit.edu
def direct_mit_edu(url):
    os.environ['WDM_LOG'] = '0'
    options = Options()
    options.add_argument('--headless')
    
    # load the webpage
    error_label = 0
    while(error_label == 0):
        try:
            driver = webdriver.Firefox(options=options)
            driver.get(url)
            time.sleep(5)
            error_label = 1
        except:
            print("Extracting content from:" + url + " failed, retrying... This might take longer than 5 minutes...")
            time.sleep(5*60)
            error_label = 0
    
    # doi
    try:
        doi = driver.find_element(By.XPATH, "//div[@class='citation-doi')]").find_element(By.TAG_NAME, "a").text.split("doi.org/")[1]
    except:
        doi = np.nan
    
    # pmid, pmcid
    pmid = np.nan
    pmcid = np.nan
    
    # title
    try:
        title = driver.find_element(By.XPATH, "//h1[@class='wi-article-title article-title-main']").text
        title = title.strip()
    except:
        title = np.nan
    
    # abstract
    try:
        abstract = ""
        elems = driver.find_element(By.XPATH, "//section[@class='abstract']").find_elements(By.XPATH, 'p')
        for elem in elems:
            abstract = abstract + elem.text + " "
        abstract = abstract.strip()
    except:
        abstract = np.nan
    
    # keywords
    keywords = np.nan

    # pdf_link
    try:
        pdf_link = driver.find_element(By.XPATH, "//a[contains(@title,'article-pdfLink']").get_attribute("href")
        pdf_link = pdf_link.strip()
    except:
        pdf_link = np.nan

    driver.quit()

    info = {
        "doi": doi,
        "pmid": pmid,
        "pmcid": pmcid,
        "title": title,
        "abstract": abstract,
        "keywords": keywords,
        "pdf_link": pdf_link
    }
    driver.quit

    return info
# --------------------start of test code--------------------
# url = "https://direct.mit.edu/jocn/article/10/6/691/3328/The-von-Restorff-Effect-in-Visual-Object"
# # url = "https://direct.mit.edu/neco/article-abstract/15/4/735/6721/Modeling-Reverse-Phi-Motion-Selective-Neurons-in?redirectedFrom=fulltext"
# # url = "https://direct.mit.edu/neco/article-abstract/24/7/1695/7782/Neural-Information-Processing-with-Feedback?redirectedFrom=fulltext"
# info = direct_mit_edu(url)
# print(info["doi"])
# print(info["pmid"])
# print(info["pmcid"])
# print(info["title"])
# print(info["abstract"])
# print(info["keywords"])
# print(info["pdf_link"])
# ---------------------end of test code---------------------

In [ ]:
# thejns.org
def thejns_org(url):
    os.environ['WDM_LOG'] = '0'
    options = Options()
    options.add_argument('--headless')
    
    # load the webpage
    error_label = 0
    while(error_label == 0):
        try:
            driver = webdriver.Firefox(options=options)
            driver.get(url)
            time.sleep(5)
            error_label = 1
        except:
            print("Extracting content from:" + url + " failed, retrying... This might take longer than 5 minutes...")
            time.sleep(5*60)
            error_label = 0
    
    # doi
    try:
        doi = driver.find_element(By.XPATH, "//a[@class='c-Button--link']/dd/span/a").text.split("doi.org/")[1]
    except:
        doi = np.nan
    
    # pmid, pmcid
    pmid = np.nan
    pmcid = np.nan

    # title
    try:
        title = driver.find_element(By.TAG_NAME, "h1").text
        title = title.strip()
    except:    
        title = np.nan
    
    # abstract
    try:
        abstract = ""
        elems = driver.find_element(By.XPATH, "//section[@class='abstract']").find_elements(By.XPATH, 'p')
        for elem in elems:
            abstract = abstract + elem.text + " "
        abstract = abstract.strip()
    except:
        abstract = np.nan
    
    # keywords
    keywords = np.nan

    # pdf_link
    try:
        pdf_link = driver.find_element(By.XPATH, "//a[@title='Download PDF']").get_attribute('href')
        pdf_link = pdf_link.strip()
    except:
        pdf_link = np.nan

    driver.quit()

    info = {
        "doi": doi,
        "pmid": pmid,
        "pmcid": pmcid,
        "title": title,
        "abstract": abstract,
        "keywords": keywords,
        "pdf_link": pdf_link
    }
    driver.quit

    return info
# --------------------start of test code--------------------
# url = "https://thejns.org/view/journals/j-neurosurg/86/1/article-p77.xml"
# url = "https://thejns.org/view/journals/j-neurosurg/41/2/article-p217.xml"
# info = thejns_org(url)
# print(info["doi"])
# print(info["pmid"])
# print(info["pmcid"])
# print(info["title"])
# print(info["abstract"])
# print(info["keywords"])
# print(info["pdf_link"])
# ---------------------end of test code---------------------

In [ ]:
# www.annualreviews.org
def www_annualreviews_org(url):
    os.environ['WDM_LOG'] = '0'
    options = Options()
    options.add_argument('--headless')
    
    # load the webpage
    error_label = 0
    while(error_label == 0):
        try:
            driver = webdriver.Firefox(options=options)
            driver.get(url)
            time.sleep(5)
            error_label = 1
        except:
            print("Extracting content from:" + url + " failed, retrying... This might take longer than 5 minutes...")
            time.sleep(5*60)
            error_label = 0
    
    # doi
    try:
        doi = driver.find_element(By.XPATH, "//div[@class='article-details]/p/a").text.split("doi.org/")[1]
    except:
        doi = np.nan
    
    # pmid, pmcid
    pmid = np.nan
    pmcid = np.nan

    # title
    try:
        title = driver.find_element(By.TAG_NAME, "h1").text
        title = title.strip()
    except:    
        title = np.nan
    
    # abstract
    try:
        abstract = ""
        elems = driver.find_element(By.XPATH, "//div[@class='abstractSection abstractInFull']").find_elements(By.XPATH, 'p')
        for elem in elems:
            abstract = abstract + elem.text + " "
        abstract = abstract.strip()
    except:
        abstract = np.nan
    
    # keywords
    try:
        keywords = ""
        elems = driver.find_element(By.XPATH, "//div[@class='hlFld-KeywordText']/p[1]/kwd-group[1]").find_elements(By.XPATH, 'a')
        for elem in elems:
            keywords = keywords + elem.text + ", "
    except:
        keywords = np.nan

    # pdf_link
    try:
        pdf_link = driver.find_element(By.XPATH, "//a[@class='btn icon-pdf']").get_attribute('href')
        pdf_link = pdf_link.strip()
    except:
        pdf_link = np.nan

    driver.quit()

    info = {
        "doi": doi,
        "pmid": pmid,
        "pmcid": pmcid,
        "title": title,
        "abstract": abstract,
        "keywords": keywords,
        "pdf_link": pdf_link
    }
    driver.quit

    return info
# --------------------start of test code--------------------
# url = "https://www.annualreviews.org/doi/10.1146/annurev.ne.02.030179.001303"
# url = "https://www.annualreviews.org/doi/10.1146/annurev.ps.23.020172.002023"
# url = "https://www.annualreviews.org/doi/10.1146/annurev.neuro.23.1.127"
# info = www_annualreviews_org(url)
# print(info["doi"])
# print(info["pmid"])
# print(info["pmcid"])
# print(info["title"])
# print(info["abstract"])
# print(info["keywords"])
# print(info["pdf_link"])
# ---------------------end of test code---------------------

In [ ]:
# aspetjournals.org
def aspetjournals_org(url):
    os.environ['WDM_LOG'] = '0'
    options = Options()
    options.add_argument('--headless')
    
    # load the webpage
    error_label = 0
    while(error_label == 0):
        try:
            driver = webdriver.Firefox(options=options)
            driver.get(url)
            time.sleep(5)
            error_label = 1
        except:
            print("Extracting content from:" + url + " failed, retrying... This might take longer than 5 minutes...")
            time.sleep(5*60)
            error_label = 0
    
    # doi
    try:
        doi = driver.find_element(By.XPATH, "//span[contains(@class, 'cite-metadata-doi')]").text.split("doi.org/")[1]
    except:
        doi = np.nan
    if doi == doi:
        doi = doi.lower()

    # pmid, pmcid
    pmid = np.nan
    pmcid = np.nan

    # title
    try:
        title = driver.find_element(By.XPATH, "//h1[@id='page-title']").text
        title = title.strip()
    except:
        title = np.nan
    
    # abstract
    try:
        abstract = ""
        elems = driver.find_element(By.XPATH, "//div[@class='section abstract']").find_elements(By.XPATH, 'p')
        for elem in elems:
            abstract = abstract + elem.text + " "
        abstract = abstract.strip()
    except:
        abstract = np.nan
    
    # keywords
    keywords = np.nan

    # pdf_link
    # try:
    #     pdf_link = driver.find_element(By.XPATH, "//a[@data-panel-name='jnl_jpet_tab_pdf']").get_attribute('href')
    #     pdf_link = pdf_link.strip()
    # except:
    #     pdf_link = np.nan
    pdf_link = np.nan

    driver.quit()

    info = {
        "doi": doi,
        "pmid": pmid,
        "pmcid": pmcid,
        "title": title,
        "abstract": abstract,
        "keywords": keywords,
        "pdf_link": pdf_link
    }
    driver.quit

    return info
# --------------------start of test code--------------------
# url = "https://jpet.aspetjournals.org/content/321/1/116.short"
# # url = "https://jpet.aspetjournals.org/content/316/2/772"
# # url = "https://jpet.aspetjournals.org/content/319/2/561"
# # url = "https://jpet.aspetjournals.org/content/325/2/629"
# info = aspetjournals_org(url)
# print(info["doi"])
# print(info["pmid"])
# print(info["pmcid"])
# print(info["title"])
# print(info["abstract"])
# print(info["keywords"])
# print(info["pdf_link"])
# ---------------------end of test code---------------------

In [ ]:
# jnm.snmjournals.org
def jnm_snmjournals_org(url):
    os.environ['WDM_LOG'] = '0'
    options = Options()
    options.add_argument('--headless')
    
    # load the webpage
    error_label = 0
    while(error_label == 0):
        try:
            driver = webdriver.Firefox(options=options)
            driver.get(url)
            time.sleep(5)
            error_label = 1
        except:
            print("Extracting content from:" + url + " failed, retrying... This might take longer than 5 minutes...")
            time.sleep(5*60)
            error_label = 0
    
    # doi
    # try:
    #     doi = driver.find_element(By.XPATH, "//span[contains(@class, 'cite-metadata-doi')]").text.split("doi.org/")[1]
    # except:
    #     doi = np.nan
    # if doi == doi:
    #     doi = doi.lower()
    doi = np.nan

    # pmid, pmcid
    pmid = np.nan
    pmcid = np.nan

    # title
    try:
        title = driver.find_element(By.XPATH, "//h1[@id='page-title']").text
        title = title.strip()
    except:
        title = np.nan
    
    # abstract
    try:
        abstract = ""
        elems = driver.find_element(By.XPATH, "//div[@class='section abstract']").find_elements(By.XPATH, 'p')
        for elem in elems:
            abstract = abstract + elem.text + " "
        abstract = abstract.strip()
    except:
        abstract = np.nan
    
    # keywords
    keywords = np.nan

    # pdf_link
    try:
        pdf_link = driver.find_element(By.XPATH, "//a[@data-panel-name='jnl_snm_tab_pdf']").get_attribute('href')
        pdf_link = pdf_link.strip()
    except:
        pdf_link = np.nan

    driver.quit()

    info = {
        "doi": doi,
        "pmid": pmid,
        "pmcid": pmcid,
        "title": title,
        "abstract": abstract,
        "keywords": keywords,
        "pdf_link": pdf_link
    }
    driver.quit

    return info
# --------------------start of test code--------------------
# url = "https://jnm.snmjournals.org/content/39/2/281.long"
# # url = "https://jnm.snmjournals.org/content/45/5/878.long"
# # url = "https://jnm.snmjournals.org/content/36/7/1275.long"
# info = jnm_snmjournals_org(url)
# print(info["doi"])
# print(info["pmid"])
# print(info["pmcid"])
# print(info["title"])
# print(info["abstract"])
# print(info["keywords"])
# print(info["pdf_link"])
# ---------------------end of test code---------------------

In [ ]:
# www.architalbiol.org
def www_architalbiol_org(url):
    os.environ['WDM_LOG'] = '0'
    options = Options()
    options.add_argument('--headless')
    
    # load the webpage
    error_label = 0
    while(error_label == 0):
        try:
            driver = webdriver.Firefox(options=options)
            driver.get(url)
            time.sleep(5)
            error_label = 1
        except:
            print("Extracting content from:" + url + " failed, retrying... This might take longer than 5 minutes...")
            time.sleep(5*60)
            error_label = 0
    
    # doi
    # try:
    #     doi = driver.find_element(By.XPATH, "//span[contains(@class, 'cite-metadata-doi')]").text.split("doi.org/")[1]
    # except:
    #     doi = np.nan
    # if doi == doi:
    #     doi = doi.lower()
    doi = np.nan

    # pmid, pmcid
    pmid = np.nan
    pmcid = np.nan

    # title
    try:
        title = driver.find_element(By.XPATH, "//div[@id='articleTitle']").find_element(By.TAG_NAME, "h3").text
        title = title.strip()
    except:
        title = np.nan
    
    # abstract
    # try:
    #     abstract = ""
    #     elems = driver.find_element(By.XPATH, "//div[@class='section abstract']").find_elements(By.XPATH, 'p')
    #     for elem in elems:
    #         abstract = abstract + elem.text + " "
    #     abstract = abstract.strip()
    # except:
    #     abstract = np.nan
    abstract = np.nan
    
    # keywords
    keywords = np.nan

    # pdf_link
    # try:
    #     pdf_link = driver.find_element(By.XPATH, "//a[@data-panel-name='jnl_snm_tab_pdf']").get_attribute('href')
    #     pdf_link = pdf_link.strip()
    # except:
    #     pdf_link = np.nan
    pdf_link = "://www.architalbiol.org/"

    driver.quit()

    info = {
        "doi": doi,
        "pmid": pmid,
        "pmcid": pmcid,
        "title": title,
        "abstract": abstract,
        "keywords": keywords,
        "pdf_link": pdf_link
    }
    driver.quit

    return info
# --------------------start of test code--------------------
# url = "http://www.architalbiol.org/index.php/aib/article/view/11423/"
# # url = "http://www.architalbiol.org/index.php/aib/article/view/140315/"
# # url = "http://www.architalbiol.org/index.php/aib/article/view/122301/"
# info = www_architalbiol_org(url)
# print(info["doi"])
# print(info["pmid"])
# print(info["pmcid"])
# print(info["title"])
# print(info["abstract"])
# print(info["keywords"])
# print(info["pdf_link"])
# ---------------------end of test code---------------------

In [ ]:
# www.ahajournals.org
def www_ahajournals_org(url):
    os.environ['WDM_LOG'] = '0'
    options = Options()
    options.add_argument('--headless')
    
    # load the webpage
    error_label = 0
    while(error_label == 0):
        try:
            driver = webdriver.Firefox(options=options)
            driver.get(url)
            time.sleep(5)
            error_label = 1
        except:
            print("Extracting content from:" + url + " failed, retrying... This might take longer than 5 minutes...")
            time.sleep(5*60)
            error_label = 0
    
    # doi
    try:
        doi = driver.find_element(By.XPATH, "//a[contains(@class, 'epub-section__doi__text')]").text.split("doi.org/")[1]
    except:
        doi = np.nan

    # pmid, pmcid
    pmid = np.nan
    pmcid = np.nan

    # title
    try:
        title = driver.find_element(By.XPATH, "//h1[@class='citation__title']").text
        title = title.strip()
    except:
        title = np.nan
    
    # abstract
    try:
        abstract = ""
        elems = driver.find_element(By.XPATH, "//div[@class='abstractSection abstractInFull']").find_elements(By.XPATH, 'p')
        for elem in elems:
            abstract = abstract + elem.text + " "
        abstract = abstract.strip()
    except:
        abstract = np.nan
    
    # keywords
    keywords = np.nan

    # pdf_link
    try:
        pdf_link = driver.find_element(By.XPATH, "//a[@class='btn btn--danger']").get_attribute('href')
        pdf_link = pdf_link.strip()
    except:
        pdf_link = np.nan

    driver.quit()

    info = {
        "doi": doi,
        "pmid": pmid,
        "pmcid": pmcid,
        "title": title,
        "abstract": abstract,
        "keywords": keywords,
        "pdf_link": pdf_link
    }
    driver.quit

    return info
# --------------------start of test code--------------------
# url = "https://www.ahajournals.org/doi/full/10.1161/01.STR.0000087786.38997.9E"
# # url = "https://www.ahajournals.org/doi/10.1161/01.STR.6.1.42"
# # url = "https://www.ahajournals.org/doi/10.1161/01.STR.32.1.107"
# # url = "https://www.ahajournals.org/doi/10.1161/01.STR.29.11.2377"
# info = www_ahajournals_org(url)
# print(info["doi"])
# print(info["pmid"])
# print(info["pmcid"])
# print(info["title"])
# print(info["abstract"])
# print(info["keywords"])
# print(info["introduction"])
# print(info["pdf_link"])
# ---------------------end of test code---------------------

In [ ]:
# pubs.acs.org
def pubs_acs_org(url):
    os.environ['WDM_LOG'] = '0'
    options = Options()
    options.add_argument('--headless')
    
    # load the webpage
    error_label = 0
    while(error_label == 0):
        try:
            driver = webdriver.Firefox(options=options)
            driver.get(url)
            time.sleep(5)
            error_label = 1
        except:
            print("Extracting content from:" + url + " failed, retrying... This might take longer than 5 minutes...")
            time.sleep(5*60)
            error_label = 0
    
    # doi
    try:
        doi = driver.find_element(By.XPATH, "//div[contains(@class, 'article_header-doiurl')]").find_element(By.TAG_NAME, "a").text.split("doi.org/")[1]
    except:
        doi = np.nan

    # pmid, pmcid
    pmid = np.nan
    pmcid = np.nan

    # title
    try:
        title = driver.find_element(By.XPATH, "//h1[@class='article_header-title']").find_element(By.TAG_NAME, "span").text
        title = title.strip()
    except:
        title = np.nan
    
    # abstract
    try:
        abstract = driver.find_element(By.XPATH, "//p[@class='articleBody_abstractText']").text
        abstract = abstract.strip()
    except:
        abstract = np.nan
    
    # keywords
    keywords = np.nan

    # pdf_link
    # try:
    #     pdf_link = driver.find_element(By.XPATH, "//a[@class='btn btn--danger']").get_attribute('href')
    #     pdf_link = pdf_link.strip()
    # except:
    #     pdf_link = np.nan
    pdf_link = np.nan

    driver.quit()

    info = {
        "doi": doi,
        "pmid": pmid,
        "pmcid": pmcid,
        "title": title,
        "abstract": abstract,
        "keywords": keywords,
        "pdf_link": pdf_link
    }
    driver.quit

    return info
# --------------------start of test code--------------------
# url = "https://pubs.acs.org/doi/10.1021/jm030384e"
# # url = "https://pubs.acs.org/doi/10.1021/jm301597s"
# # url = "https://pubs.acs.org/doi/10.1021/acs.molpharmaceut.8b01209"
# info = pubs_acs_org(url)
# print(info["doi"])
# print(info["pmid"])
# print(info["pmcid"])
# print(info["title"])
# print(info["abstract"])
# print(info["keywords"])
# print(info["pdf_link"])
# ---------------------end of test code---------------------

In [ ]:
# www.thieme-connect.de
def www_thieme_connect_de(url):
    os.environ['WDM_LOG'] = '0'
    options = Options()
    options.add_argument('--headless')
    
    # load the webpage
    error_label = 0
    while(error_label == 0):
        try:
            driver = webdriver.Firefox(options=options)
            driver.get(url)
            time.sleep(5)
            error_label = 1
        except:
            print("Extracting content from:" + url + " failed, retrying... This might take longer than 5 minutes...")
            time.sleep(5*60)
            error_label = 0
    
    # doi
    try:
        doi = driver.find_element(By.XPATH, "//div[contains(@class, 'doi')]").find_element(By.TAG_NAME, "a").text.split("DOI: ")[1]
    except:
        doi = np.nan

    # pmid, pmcid
    pmid = np.nan
    pmcid = np.nan

    # title
    try:
        title = driver.find_element(By.TAG_NAME, "h1").text
        title = title.strip()
    except:
        title = np.nan
    
    # abstract
    try:
        abstract = driver.find_element(By.XPATH, "//h3[contains(text(),'Abstract')]").find_element(By.XPATH, "followingslibing::p").text
        abstract = abstract.strip()
    except:
        abstract = np.nan
    
    # keywords
    try:
        keywords = driver.find_element(By.XPATH, "//div[@class='articleKeywords')]").find_element(By.TAG_NAME, "p").text
        keywords = keywords.strip()
    except:
        keywords = np.nan
    # keywords = np.nan

    # pdf_link
    # try:
    #     pdf_link = driver.find_element(By.XPATH, "//a[@class='btn btn--danger']").get_attribute('href')
    #     pdf_link = pdf_link.strip()
    # except:
    #     pdf_link = np.nan
    pdf_link = "://www.thieme-connect.de/"

    driver.quit()

    info = {
        "doi": doi,
        "pmid": pmid,
        "pmcid": pmcid,
        "title": title,
        "abstract": abstract,
        "keywords": keywords,
        "pdf_link": pdf_link
    }
    driver.quit

    return info
# --------------------start of test code--------------------
# url = "https://www.thieme-connect.de/products/ejournals/abstract/10.1055/s-2007-973495"
# # url = "https://www.thieme-connect.de/products/ejournals/abstract/10.1055/s-0031-1299170"
# info = www_thieme_connect_de(url)
# print(info["doi"])
# print(info["pmid"])
# print(info["pmcid"])
# print(info["title"])
# print(info["abstract"])
# print(info["keywords"])
# print(info["pdf_link"])
# ---------------------end of test code---------------------

In [ ]:
# pubs.asahq.org
def pubs_asahq_org(url):
    os.environ['WDM_LOG'] = '0'
    options = Options()
    options.add_argument('--headless')
    
    # load the webpage
    error_label = 0
    while(error_label == 0):
        try:
            driver = webdriver.Firefox(options=options)
            driver.get(url)
            time.sleep(5)
            error_label = 1
        except:
            print("Extracting content from:" + url + " failed, retrying... This might take longer than 5 minutes...")
            time.sleep(5*60)
            error_label = 0
    
    # doi
    try:
        doi = driver.find_element(By.XPATH, "//div[contains(@class, 'citation-doi')]").find_element(By.TAG_NAME, "a").text.split("doi.org/")[1]
    except:
        doi = np.nan

    # pmid, pmcid
    pmid = np.nan
    pmcid = np.nan

    # title
    try:
        title = driver.find_element(By.XPATH, "//h1[contains(@class,'article-title')]").text
        title = title.strip()
    except:
        title = np.nan
    
    # abstract
    try:
        abstract = driver.find_element(By.XPATH, "//section[@class='abstract']").text
        abstract = abstract.strip()
    except:
        abstract = np.nan
    
    # keywords
    try:
        keywords = ""
        elems = driver.find_element(By.XPATH, "//div[@class='content-metadata-topics')]").find_elements(By.TAG_NAME, "a")
        for elem in elems:
            keywords = keywords + elem.text + "; "
        keywords = keywords.strip()
    except:
        keywords = np.nan
    # keywords = np.nan

    # pdf_link
    # try:
    #     pdf_link = driver.find_element(By.XPATH, "//a[contains(@class,'article-pdfLink']").get_attribute('href')
    #     pdf_link = pdf_link.strip()
    # except:
    #     pdf_link = np.nan
    pdf_link = "://www.thieme-connect.de/"

    driver.quit()

    info = {
        "doi": doi,
        "pmid": pmid,
        "pmcid": pmcid,
        "title": title,
        "abstract": abstract,
        "keywords": keywords,
        "pdf_link": pdf_link
    }
    driver.quit

    return info
# --------------------start of test code--------------------
# url = "https://pubs.asahq.org/anesthesiology/article/116/2/372/13001/Ketamine-induced-Neuroapoptosis-in-the-Fetal-and"
# # url = "https://pubs.asahq.org/anesthesiology/article/66/1/39/29057/The-Effects-of-Dextrose-Infusion-and-Head-Position"
# # url = "https://pubs.asahq.org/anesthesiology/article/98/5/1101/40355/Neural-Mechanism-of-Propofol-Anesthesia-in-Severe"
# info = pubs_asahq_org(url)
# print(info["doi"])
# print(info["pmid"])
# print(info["pmcid"])
# print(info["title"])
# print(info["abstract"])
# print(info["keywords"])
# print(info["pdf_link"])
# ---------------------end of test code---------------------

In [ ]:
# www.ingentaconnect.com
def www_ingentaconnect_com(url):
    os.environ['WDM_LOG'] = '0'
    options = Options()
    options.add_argument('--headless')
    
    # load the webpage
    error_label = 0
    while(error_label == 0):
        try:
            driver = webdriver.Firefox(options=options)
            driver.get(url)
            time.sleep(5)
            error_label = 1
        except:
            print("Extracting content from:" + url + " failed, retrying... This might take longer than 5 minutes...")
            time.sleep(5*60)
            error_label = 0
    
    # doi
    # try:
    #     doi = driver.find_element(By.XPATH, "//div[contains(@class, 'citation-doi')]").find_element(By.TAG_NAME, "a").text.split("doi.org/")[1]
    # except:
    #     doi = np.nan
    doi = np.nan

    # pmid, pmcid
    pmid = np.nan
    pmcid = np.nan

    # title
    try:
        title = driver.find_element(By.XPATH, "//h1[contains(@class,'abstract-heading')]").text
        title = title.strip()
    except:
        title = np.nan
    
    # abstract
    try:
        abstract = driver.find_element(By.XPATH, "//div[@class='Abst']").text
        abstract = abstract.strip()
    except:
        abstract = np.nan
    
    # keywords
    try:
        keywords = ""
        elems = driver.find_element(By.XPATH, "//div[@class='content-metadata-topics')]").find_elements(By.TAG_NAME, "a")
        for elem in elems:
            keywords = keywords + elem.text + "; "
        keywords = keywords.strip()
    except:
        keywords = np.nan
    # keywords = np.nan

    # pdf_link
    # try:
    #     pdf_link = driver.find_element(By.XPATH, "//ul[contains(@class,'right-col-download contain']").get_attribute('href')
    #     pdf_link = pdf_link.strip()
    # except:
    #     pdf_link = np.nan
    pdf_link = "://www.ingentaconnect.com/"

    driver.quit()

    info = {
        "doi": doi,
        "pmid": pmid,
        "pmcid": pmcid,
        "title": title,
        "abstract": abstract,
        "keywords": keywords,
        "pdf_link": pdf_link
    }
    driver.quit

    return info
# --------------------start of test code--------------------
# url = "https://www.ingentaconnect.com/content/aalas/cm/2000/00000050/00000002/art00006;jsessionid=9jxpglps7nq4.x-ic-live-03"
# info = www_ingentaconnect_com(url)
# print(info["doi"])
# print(info["pmid"])
# print(info["pmcid"])
# print(info["title"])
# print(info["abstract"])
# print(info["keywords"])
# print(info["pdf_link"])
# ---------------------end of test code---------------------

In [ ]:
# ujms.net


In [ ]:
# journals.biologists.com

In [ ]:
# www.microbiologyresearch.org

In [ ]:
# journals.aps.org

In [ ]:
# www.imrpress.com

In [ ]:
# www.researchsquare.com

In [ ]:
# ieeexplore.ieee.org

In [ ]:
# papers.ssrn.com